# 使用tree-sitter用于csharp代码分析

In [1]:
import os
import pickle
from typing import *

## 1. 安装csharp语法扩展

```bash
pip install tree-sitter-c-sharp
```

In [2]:
from tree_sitter import Language, Parser

In [11]:
import tree_sitter_c_sharp

In [27]:
cslang = tree_sitter.Language(tree_sitter_c_sharp.language())

In [28]:
parser = Parser(cslang)

In [29]:
# 解析 C# 代码
with open("../data/ImageSharp-main/src/ImageSharp/Color/Color.cs", "rb") as f:
    tree = parser.parse(f.read())

In [30]:
tree

In [33]:
def traverse_tree(node, indent=0):
    print(' ' * indent + f'{node.type} [{node.start_point} - {node.end_point}]')
    for child in node.children:
        traverse_tree(child, indent + 2)

In [34]:
traverse_tree(tree.root_node)

compilation_unit [Point(row=0, column=0) - Point(row=339, column=0)]
  comment [Point(row=0, column=0) - Point(row=0, column=28)]
  comment [Point(row=1, column=0) - Point(row=1, column=47)]
  using_directive [Point(row=3, column=0) - Point(row=3, column=22)]
    using [Point(row=3, column=0) - Point(row=3, column=5)]
    qualified_name [Point(row=3, column=6) - Point(row=3, column=21)]
      identifier [Point(row=3, column=6) - Point(row=3, column=12)]
      . [Point(row=3, column=12) - Point(row=3, column=13)]
      identifier [Point(row=3, column=13) - Point(row=3, column=21)]
    ; [Point(row=3, column=21) - Point(row=3, column=22)]
  using_directive [Point(row=4, column=0) - Point(row=4, column=38)]
    using [Point(row=4, column=0) - Point(row=4, column=5)]
    qualified_name [Point(row=4, column=6) - Point(row=4, column=37)]
      qualified_name [Point(row=4, column=6) - Point(row=4, column=20)]
        identifier [Point(row=4, column=6) - Point(row=4, column=12)]
        . [Poi

## 2. 解析csharp项目结构

1. 引用声明，区分外部库以及本地项目文件引用
2. 注释，包括代码文件注释以及类的docstring
3. 类以及类的成员，构建两个层级的对象`class-->attribute`之后分析以attribute为基本粒度，得到结果之后与类注释一起作为类功能分析的输入
4. 根据引用声明构建项目文件之间的依赖关系